In [1]:
#
# 인수를 순회할 때는 방어적으로 하자
#
# 파라미터로 객체의 리스트를 여러 번 순회해야할 때가 있다.
# 예를 들어 미국 텍사스주의 여행자 수를 분석하고 싶다고 해보자.
# 데이터 집합은 각 도시의 방문자 수(연도별 백만 명 단위)라고 하자.
# 각 도시에서 전체 여행자 중 몇 퍼센트를 받아들이는지 알고 싶을 것이다.

# 이런 작업을 하려면 정규화 함수가 필요하다. 정규화 함수에서는 입력을 합산해서 연도별 총 여행자 수를 구한다.
# 각 도시의 방문자 수를 전체 방문자 수로 나누어 각 도시가 전체에서 차지하는 비중을 알아낸다.
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

visits = [15, 35, 80]
percentage = normalize(visits)
print(percentage)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [2]:
# 방문 리스트를 확대하려면, 텍사스 주의 모든 도시가 들어있는 파일에서 데이터를 읽을 수 있어야한다.
# 이 작업을 수행하는 제너레이터를 정의할텐데, 그러면 나중에 같은 함수를 재사용하여 더 큰 데이터 세트인 "전 세계 여행자 수"를 계산할 수 있기 때문이다.
def read_visit(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

In [3]:
# 하지만 이 상태에서 함수를 실행해봐야 아무것도 나오지 않는다.
it = read_visit('my_numbers.txt')
percentage2 = normalize(it)
print(percentage2)

[]


In [4]:
# 그 이유인 즉슨, 이터레이터는 결과를 딱 한번만 생성한다.
# 이미 'StopIteration'예외를 일으킨 이터레이터나 제너레이터를 순회하면, 어떤 결과도 얻을 수 없다.
it = read_visit('my_numbers.txt')
print(list(it))
print(list(it)) # 이미 위에서 결과를 한번 생성했기 때문에 2번 째에는 빈값이 리턴된다.

[15, 22, 30]
[]


In [5]:
# 이미 한번 함수가 호출이 되서 2번째부터는 빈값이 나오더라도, 결코 에러는 나지 않는다. 그래서 더 헷깔릴 수 있다.
# 이러한 문제를 해결하려면, 입력 이터레이터를 명시적으로 소진한 후에, 전체 콘텐츠의 복사본을 리스트에 저장해야한다.
# 그러면 리스트 버전의 데이터를 필요한 만큼 순회할 수 있다.
def normalize_copy(numbers):
    numbers = list(numbers)
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100*value/total
        result.append(percent)
    return result

it = read_visit('my_numbers.txt')
percentages = normalize_copy(it)
print(percentages)

[22.388059701492537, 32.83582089552239, 44.776119402985074]


In [6]:
# 하지만 이 방법을 사용하면 굳이 이터레이터를 사용하는 의미가 없다.
# 이 문제점을 피하는 방법은 호출될 때마다 새 이터레이터를 반환하는 함수를 받게 만드는 것이다.
def normalize_func(get_iter):
    total = sum(get_iter())
    result = []
    for value in get_iter():
        percent = 100 * value / total
        result.append(percent)
    return result

In [10]:
# normalize_func을 사용하려면 제너레이터를 호출해서 매번 새 이터레이터를 생성하는 람다 표현식을 넘겨주면 된다.
percentages3 = normalize_func(lambda: read_visit('my_numbers.txt'))
print(percentages3)

[22.388059701492537, 32.83582089552239, 44.776119402985074]


In [23]:
# 정삭적으로 작동하긴 하지만, 람다 함수를 넘겨주는 방법은 별로 좋지 않다.
# 더 좋은 방법은 이터레이터 프로토콜을 구현한 새 컨테이너 클래스를 제공하는 것이다.
# 이터레이터 프로토콜은 파이썬의 for 루프와 관련 표현식이 컨테이너 타입의 콘텐츠를 탐색하는 방법을 나타낸다.(뭔 개소리냐...)

# 1. for x in foo 와 같은 구문은 실제로 iter(foo)를 호출한다.
# 2. 내장함수 iter는 매직 메소드인 foo.__iter__를 호출한다.
# 3. __iter__ 메소드는 이터레이터 객체를 리턴해야한다. (next 함수를 쓸 수 있다면 이터레이터라 볼 수 있다.)
# 4. for 루프는 이터레이터를 모두 소진할 때 까지 next 함수를 호출한다.

# 존나 복잡하지만 우리도 __iter__만 좀 쓸줄 알면 존나 간지나는 클래스를 뽑아낼 수 있다.
class ReadVisits:
    def __init__(self, data_path):
        self.data_path = data_path
        
    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)

visits = ReadVisits('my_numbers.txt')
print(visits)
percent = normalize(visits)
print(percent)

[22.388059701492537, 32.83582089552239, 44.776119402985074]
